In [ ]:

# Instalar librerías
try:
    import timesfm
    print("✅ La librería 'timesfm' ya está instalada.")
except ImportError:
    print("Instalando 'timesfm'. Esto puede tardar unos minutos...")
    !pip install --quiet "timesfm[torch]"

# Importar librerías generales
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler  

try:
    from google.colab import drive
    drive.mount('/content/drive')
except: pass

# Importaciones específicas del modelo
try:
    from huggingface_hub import snapshot_download
    from timesfm import TimesFm, TimesFmHparams
    from timesfm.pytorch_patched_decoder import PatchedTimeSeriesDecoder
    from finetuning.finetuning_torch import FinetuningConfig, TimesFMFinetuner
    print("✅ Módulos importados correctamente.")
except ImportError as e:
    print(f"🛑 ERROR: No se pudo importar un módulo necesario: {e}")

class Config:
    BASE_PATH = "/content/drive/My Drive/Colab Notebooks/CHRONOS BOLT"
    DATA_PATH = os.path.join(BASE_PATH, "data_chronos.csv")
    MODEL_SAVE_PATH = os.path.join(BASE_PATH, "timesfm_finetuned_scaled.pth")  
    PREDICTION_SAVE_PATH = os.path.join(BASE_PATH, "kaggle_submission_scaled.csv")  
    CONTEXT_LEN = 15
    HORIZON_LEN = 2
    NUM_EPOCHS = 5
    BATCH_SIZE = 256
    LEARNING_RATE = 1e-4
config = Config()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Configuración cargada. Usando dispositivo: {device}")

class RollingWindowsDataset(Dataset):
    # Modificado para aceptar el nombre de la columna de valores
    def __init__(self, df, context_len, horizon_len, value_col='value'):
        self.samples = []
        df = df.sort_values(["item_id", "timestamp"])
        for _, group in df.groupby("item_id"):
            # Usa la columna especificada (ej. 'value' o 'scaled_value')
            series_values = group[value_col].values
            n = len(series_values)
            for i in range(n - (context_len + horizon_len) + 1):
                self.samples.append((series_values[i : i + context_len], series_values[i + context_len : i + context_len + horizon_len]))
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        c, t = self.samples[idx]
        return (torch.tensor(c, dtype=torch.float32), torch.zeros_like(torch.tensor(c)), torch.tensor([1]), torch.tensor(t, dtype=torch.float32))

def run_pipeline():
    # Carga de datos 

    try:
        main_df = pd.read_csv(config.DATA_PATH, parse_dates=["timestamp"])
        print(f"✅ Datos cargados.")
    except FileNotFoundError:
        print(f"🛑 ERROR: No se encontró el archivo de datos. Verifica la ruta: {config.DATA_PATH}"); return

    # --- [NUEVO] Paso 3.5: Escalado de Datos por Producto ---
    print("\n--- [NUEVO] Escalando datos por producto ---")
    scalers = {}
    scaled_values = []

    for item_id, group in tqdm(main_df.groupby('item_id'), desc="Escalando productos"):
        scaler = StandardScaler()
        # Ajustamos y transformamos la columna 'value'
        scaled_group = scaler.fit_transform(group[['value']])
        scaled_values.append(scaled_group)
        # Guardamos el scaler para este producto específico
        scalers[item_id] = scaler

    # Añadimos los valores escalados como una nueva columna al DataFrame
    main_df['scaled_value'] = np.vstack(scaled_values)
    print("✅ Datos escalados y scalers guardados.")

    #  Preparación del Modelo Base  
    print("\n--- [Paso 4/6] Preparando el modelo TimesFM ---")
    TimesFm.load_from_checkpoint = lambda self, checkpoint: None
    repo_id = "google/timesfm-2.0-500m-pytorch"; ckpt_dir = snapshot_download(repo_id=repo_id)
    weights_path = os.path.join(ckpt_dir, "torch_model.ckpt")
    hparams = TimesFmHparams(context_len=config.CONTEXT_LEN, horizon_len=config.HORIZON_LEN, input_patch_len=config.CONTEXT_LEN, output_patch_len=config.HORIZON_LEN, backend="pytorch")
    model_config = TimesFm(hparams=hparams, checkpoint=None)._model_config
    model = PatchedTimeSeriesDecoder(model_config)
    weights = torch.load(weights_path, map_location=device); state = model.state_dict()
    filtered_weights = {k: v for k, v in weights.items() if k in state and state[k].shape == v.shape}
    model.load_state_dict(filtered_weights, strict=False); model.to(device)
    print(f"✅ Modelo base creado.")

    #  Fine-Tuning  
    print("\n--- [Paso 5/6] Proceso de Fine-Tuning ---")
    if os.path.exists(config.MODEL_SAVE_PATH):
        print(f"✅ Modelo ajustado ya existe. Cargando pesos desde: {config.MODEL_SAVE_PATH}")
        model.load_state_dict(torch.load(config.MODEL_SAVE_PATH, map_location=device))
    else:
        print("Iniciando fine-tuning con datos escalados...")
        # Le indicamos al Dataset que use la nueva columna 'scaled_value'
        finetuning_dataset = RollingWindowsDataset(main_df, config.CONTEXT_LEN, config.HORIZON_LEN, value_col='scaled_value')

        finetuner_config = FinetuningConfig(batch_size=config.BATCH_SIZE, num_epochs=config.NUM_EPOCHS, learning_rate=config.LEARNING_RATE, val_check_interval=0.0)
        finetuner = TimesFMFinetuner(model, finetuner_config)
        for epoch in tqdm(range(config.NUM_EPOCHS), desc="Épocas de Fine-Tuning"):
            finetuner.finetune(train_dataset=finetuning_dataset, val_dataset=finetuning_dataset)
        print(f"✅ Fine-tuning completado. Guardando modelo en: {config.MODEL_SAVE_PATH}")
        torch.save(model.state_dict(), config.MODEL_SAVE_PATH)

    #  Predicción y Desescalado 

    if os.path.exists(config.PREDICTION_SAVE_PATH):
        print(f"✅ Archivo de predicciones ya existe.")
        final_predictions = pd.read_csv(config.PREDICTION_SAVE_PATH)
    else:
        print("Generando predicciones finales (M+2)...")
        model.eval(); predictions = []
        for pid, group in tqdm(main_df.groupby("item_id"), desc="Prediciendo por producto"):
            # El contexto para la predicción debe venir de los datos escalados
            scaled_series_values = group['scaled_value'].values
            context = scaled_series_values[-config.CONTEXT_LEN:]
            context_tensor = torch.tensor(context, dtype=torch.float32).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(context_tensor, torch.zeros_like(context_tensor), torch.ones(1, dtype=torch.long).to(device))

            # La predicción del modelo está en la escala normalizada
            scaled_pred = output[0, 0, 1, 0].item()

            # Recuperamos el scaler correcto para este producto
            scaler = scalers[pid]

            # Desescalamos la predicción para devolverla a su magnitud original
            descaled_pred = scaler.inverse_transform([[scaled_pred]])[0, 0]

            predictions.append({"product_id": pid, "tn": descaled_pred})

        final_predictions = pd.DataFrame(predictions)
        final_predictions.to_csv(config.PREDICTION_SAVE_PATH, index=False, float_format="%.5f")
        print(f"✅ Archivo de predicciones guardado.")

    print("\n--- Resultados (en escala original) ---")
    print(final_predictions.head())
    print("\n🎉 Proceso completado exitosamente.")

# Ejecutar el pipeline completo
run_pipeline()

--- [Paso 1/6] Configurando el entorno de Colab ---
✅ La librería 'timesfm' ya está instalada.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Módulos importados correctamente.

--- [Paso 2/6] Cargando configuración del script ---
✅ Configuración cargada. Usando dispositivo: cuda

--- [Paso 3/6] Cargando y procesando datos ---
✅ Datos cargados.

--- [NUEVO] Escalando datos por producto ---


Escalando productos:   0%|          | 0/780 [00:00<?, ?it/s]

✅ Datos escalados y scalers guardados.

--- [Paso 4/6] Preparando el modelo TimesFM ---


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Modelo base creado.

--- [Paso 5/6] Proceso de Fine-Tuning ---
Iniciando fine-tuning con datos escalados...


Épocas de Fine-Tuning:   0%|          | 0/5 [00:00<?, ?it/s]

✅ Fine-tuning completado. Guardando modelo en: /content/drive/My Drive/Colab Notebooks/CHRONOS BOLT/timesfm_finetuned_scaled.pth

--- [Paso 6/6] Proceso de Predicción y Desescalado ---
Generando predicciones finales (M+2)...


Prediciendo por producto:   0%|          | 0/780 [00:00<?, ?it/s]

✅ Archivo de predicciones guardado.

--- Resultados (en escala original) ---
   product_id           tn
0       20001  1266.969705
1       20002  1026.404922
2       20003   693.279040
3       20004   556.889817
4       20005   566.027349

🎉 Proceso completado exitosamente.
